In [33]:
%pip install -q -U langchain
%pip install -q -U langchain-google-genai
%pip install -q -U langgraph

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

In [35]:
from google.colab import userdata
google_api_key = userdata.get("GEMINI_API_KEY")

In [36]:
llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=google_api_key)

In [37]:
llm_response: AIMessage = llm.invoke("How powerful are starship raptor engines? explain to a 5 yr")
llm_response

AIMessage(content='Imagine a tiny little firecracker. It makes a loud "pop" when you light it, right? Now imagine a firecracker that\'s a million times bigger! That\'s kind of like the Raptor engine on a Starship. It burns fuel really fast and makes a huge push, like a super-powerful rocket! That push helps the Starship fly really high into space, just like a big, strong superhero! \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'LOW', 'blocked': False}]}, id='run-2be032ad-870b-4bef-a8b8-cc165a311e8c-0', usage_metadata={'input_tokens': 15, 'output_toke

In [38]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

In [40]:
prompt = """Classify the following text into one of the categories: News,\
Blog, Research, or Other.\n\nText:{text}\n\nCategory:"""

In [41]:
from typing_extensions import TypedDict

def TextAnalysisState(TypedDict):
  text: str
  classification: str
  entities: str
  summary: str

In [42]:
def classification_node(state: TextAnalysisState):
  pass

In [43]:
def entity_extraction_node(state: TextAnalysisState):
  pass

In [44]:
def summarization_node(state: TextAnalysisState):
  pass